In [14]:
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import os
from pathlib import Path
from typing import Union, List, Tuple, Optional

def display_image(
        image_path: Union[str, Path],
        title: Optional[str] = None,
        figsize: Tuple[int, int] = (10, 8),
        show_info: bool = True
):
    """
    단일 이미지를 주피터 노트북에 출력합니다.

    Args:
        image_path: 이미지 파일 경로
        title: 이미지 제목 (None이면 파일명 사용)
        figsize: 출력 크기 (width, height)
        show_info: 이미지 정보 출력 여부
    """
    try:
        # 이미지 로드
        if isinstance(image_path, str):
            image_path = Path(image_path)

        if not image_path.exists():
            print(f"Error: Image not found at {image_path}")
            return

        # PIL로 이미지 로드
        img = Image.open(image_path)
        img_array = np.array(img)

        # 제목 설정
        if title is None:
            title = image_path.name

        # 이미지 출력
        plt.figure(figsize=figsize)
        plt.imshow(img_array)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.axis('off')  # 축 제거

        # 이미지 정보 출력
        if show_info:
            print(f"File: {image_path.name}")
            print(f"Size: {img.size} (W x H)")
            print(f"Mode: {img.mode}")
            print(f"Format: {img.format}")
            if hasattr(img, 'info') and img.info:
                print(f"Info: {img.info}")

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

def display_multiple_images(
        image_paths: List[Union[str, Path]],
        titles: Optional[List[str]] = None,
        cols: int = 3,
        figsize: Tuple[int, int] = (15, 10),
        show_info: bool = False
):
    """
    여러 이미지를 격자 형태로 출력합니다.

    Args:
        image_paths: 이미지 파일 경로 리스트
        titles: 각 이미지의 제목 리스트
        cols: 열 개수
        figsize: 전체 출력 크기
        show_info: 이미지 정보 출력 여부
    """
    n_images = len(image_paths)
    rows = (n_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=figsize)

    # axes를 1차원 배열로 변환
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()

    for i in range(n_images):
        try:
            # 이미지 로드
            image_path = Path(image_paths[i])
            img = Image.open(image_path)
            img_array = np.array(img)

            # 제목 설정
            if titles and i < len(titles):
                title = titles[i]
            else:
                title = image_path.name

            # 이미지 출력
            axes[i].imshow(img_array)
            axes[i].set_title(title, fontsize=12)
            axes[i].axis('off')

            # 이미지 정보 출력
            if show_info:
                print(f"{i+1}. {image_path.name}: {img.size}")

        except Exception as e:
            axes[i].text(0.5, 0.5, f'Error loading\n{image_paths[i]}',
                         ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')

    # 빈 subplot 제거
    for i in range(n_images, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

def display_image_with_bbox(
        image_path: Union[str, Path],
        bboxes: List[dict],
        title: Optional[str] = None,
        figsize: Tuple[int, int] = (12, 8),
        show_labels: bool = True
):
    """
    바운딩 박스와 함께 이미지를 출력합니다.

    Args:
        image_path: 이미지 파일 경로
        bboxes: 바운딩 박스 리스트 [{'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2, 'category': 'car'}, ...]
        title: 이미지 제목
        figsize: 출력 크기
        show_labels: 라벨 표시 여부
    """
    try:
        # 이미지 로드
        image_path = Path(image_path)
        img = Image.open(image_path)
        img_array = np.array(img)

        # 제목 설정
        if title is None:
            title = f"{image_path.name} ({len(bboxes)} objects)"

        # 이미지 출력
        plt.figure(figsize=figsize)
        plt.imshow(img_array)

        # 바운딩 박스 그리기
        colors = plt.cm.tab10(np.linspace(0, 1, 10))
        category_colors = {}

        for bbox in bboxes:
            x1, y1, x2, y2 = bbox['x1'], bbox['y1'], bbox['x2'], bbox['y2']
            category = bbox.get('category', 'unknown')

            # 카테고리별 색상 할당
            if category not in category_colors:
                color_idx = len(category_colors) % len(colors)
                category_colors[category] = colors[color_idx]

            color = category_colors[category]

            # 바운딩 박스 그리기
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                 linewidth=2, edgecolor=color, facecolor='none')
            plt.gca().add_patch(rect)

            # 라벨 표시
            if show_labels:
                plt.text(x1, y1-5, category, fontsize=10, color=color,
                         bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))

        plt.title(title, fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # 카테고리별 개수 출력
        if bboxes:
            from collections import Counter
            category_counts = Counter([bbox.get('category', 'unknown') for bbox in bboxes])
            print("Object counts:")
            for category, count in category_counts.items():
                print(f"  {category}: {count}")

    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

def browse_directory_images(
        directory: Union[str, Path],
        extensions: List[str] = ['.jpg', '.jpeg', '.png', '.bmp'],
        max_images: int = 20,
        cols: int = 4,
        figsize: Tuple[int, int] = (16, 12)
):
    """
    디렉토리의 이미지들을 브라우징합니다.

    Args:
        directory: 이미지 디렉토리 경로
        extensions: 지원할 이미지 확장자
        max_images: 최대 표시할 이미지 수
        cols: 열 개수
        figsize: 출력 크기
    """
    directory = Path(directory)

    if not directory.exists():
        print(f"Directory not found: {directory}")
        return

    # 이미지 파일 찾기
    image_files = []
    for ext in extensions:
        image_files.extend(directory.glob(f"*{ext}"))
        image_files.extend(directory.glob(f"*{ext.upper()}"))

    image_files = sorted(image_files)[:max_images]

    if not image_files:
        print(f"No image files found in {directory}")
        return

    print(f"Found {len(image_files)} images in {directory}")
    display_multiple_images(image_files, cols=cols, figsize=figsize)

# 간단한 사용 예시
def quick_show(image_path: Union[str, Path], size: str = 'medium'):
    """
    빠른 이미지 출력을 위한 간단한 함수

    Args:
        image_path: 이미지 경로
        size: 'small', 'medium', 'large'
    """
    size_map = {
        'small': (6, 4),
        'medium': (10, 8),
        'large': (15, 12)
    }

    figsize = size_map.get(size, (10, 8))
    display_image(image_path, figsize=figsize, show_info=False)

In [ ]:
"""SHIFT dataset example using PyTorch."""

import os
import sys

import torch
from torch.utils.data import DataLoader


from shift_dev import SHIFTDataset
from shift_dev.types import Keys
from shift_dev.utils.backend import ZipBackend


dataset = SHIFTDataset(
    data_root="./data/SHIFT/",
    split="val",
    keys_to_load=[
        Keys.images,                # note: images, shape (1, 3, H, W), uint8 (RGB)
        Keys.intrinsics,            # note: camera intrinsics, shape (3, 3)
        Keys.boxes2d,               # note: 2D boxes in image coordinate, (x1, y1, x2, y2)
        Keys.boxes2d_classes,       # note: class indices, shape (num_boxes,)
        Keys.boxes2d_track_ids,     # note: object ids, shape (num_ins,)
    ],
    views_to_load=["front"],
    shift_type="discrete",          # also supports "continuous/1x", "continuous/10x", "continuous/100x"
    backend=ZipBackend(),           # also supports HDF5Backend(), FileBackend()
    verbose=True,
)

dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
)

# Print the dataset size
print(f"Total number of samples: {len(dataset)}.")

# Print the tensor shape of the first batch.
print('\n')
for i, batch in enumerate(dataloader):
    print(f"Batch {i}:\n")
    print(f"{'Item':20} {'Shape':35} {'Min':10} {'Max':10}")
    print("-" * 80)
    for k, data in batch["front"].items():
        if isinstance(data, torch.Tensor):
            print(f"{k:20} {str(data.shape):35} {data.min():10.2f} {data.max():10.2f}")
        else:
            print(f"{k:20} {data}")
    break

# Print the sample indices within a video.
# The video indices groups frames based on their video sequences. They are useful for training on videos.
print('\n')
video_to_indices = dataset.video_to_indices
for video, indices in video_to_indices.items():
    print(f"Video name: {video}")
    print(f"Sample indices within a video: {indices}")
    break

In [ ]:
dataset[0]['front']

In [ ]:
from transformers import RTDetrConfig

In [ ]:
RTDetrConfig.from_pretrained("PekingU/rtdetr_r50vd").num_labels

In [34]:
from json import load, dump

In [ ]:
data = load(open("./data/SHIFT/discrete/images/train/front/det_2d.json"))

In [15]:
data.keys()

dict_keys(['frames', 'config'])

In [16]:
data['config']

{'imageSize': {'width': 1280, 'height': 800},
 'categories': [{'name': 'pedestrian'},
  {'name': 'car'},
  {'name': 'truck'},
  {'name': 'bus'},
  {'name': 'motorcycle'},
  {'name': 'bicycle'}]}

In [19]:
display_image("./data/SHIFT/discrete/images/train/front/0003-17fb/00000000_img_front.jpg")

Error: Image not found at data/SHIFT/discrete/images/train/front/0003-17fb/00000000_img_front.jpg


In [18]:
data['frames'][0]

{'name': '00000000_img_front.jpg',
 'videoName': '0003-17fb',
 'intrinsics': {'focal': [640.0, 640.0], 'center': [640.0, 400.0], 'skew': 0},
 'extrinsics': {'location': [32.49589920043945,
   -67.89702606201172,
   1.540658950805664],
  'rotation': [0.11117553182098182, -0.14815363057230854, 179.9752524025178]},
 'attributes': {'weather_coarse': 'rainy',
  'timeofday_coarse': 'night',
  'weather_fine': 'heavy rain',
  'timeofday_fine': 'night',
  'view': 'front',
  'town': '10HD',
  'sun_altitude_angle': '-2',
  'cloudiness': '100.0',
  'precipitation': '100.0',
  'precipitation_deposits': '100.0',
  'wind_intensity': '100.0',
  'sun_azimuth_angle': '0.0',
  'fog_density': '7.0',
  'fog_distance': '0.75',
  'wetness': '0.0',
  'fog_falloff': '0.1'},
 'frameIndex': 0,
 'labels': [{'id': '11367',
   'attributes': {'type': 'vehicle.volkswagen.t2_2021'},
   'category': 'bus',
   'box2d': {'x1': 272.0, 'y1': 391.0, 'x2': 377.0, 'y2': 448.0}},
  {'id': '11343',
   'attributes': {'type': 'veh

In [ ]:
from collections import Counter

In [20]:
towns = [d['attributes']['town'] for d in data['frames']]
cc = Counter(towns)
towns, cc

(['10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '10HD',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '02',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '07',
  '0

In [21]:
weather_coarse = [d['attributes']['weather_coarse'] for d in data['frames']]
wet = Counter(weather_coarse)
weather_coarse, wet

(['rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'rainy',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',
  'clear',

In [22]:
timeofday_coarse = [d['attributes']['timeofday_coarse'] for d in data['frames']]
timee = Counter(timeofday_coarse)
timeofday_coarse, timee

(['night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'night',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'daytime',
  'dayti

In [26]:
clear_daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'clear' and d['attributes']['timeofday_coarse'] == 'daytime'])
clear_night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'clear' and d['attributes']['timeofday_coarse'] == 'night'])
clear_dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'clear' and d['attributes']['timeofday_coarse'] == 'dawn/dusk'])
print(f"Clear weather datasets - Daytime: {len(clear_daytime['frames'])}, Night: {len(clear_night['frames'])}, Dawn: {len(clear_dawn['frames'])}")

cloudy_daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'cloudy' and d['attributes']['timeofday_coarse'] == 'daytime'])
cloudy_night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'cloudy' and d['attributes']['timeofday_coarse'] == 'night'])
cloudy_dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'cloudy' and d['attributes']['timeofday_coarse'] == 'dawn/dusk'])
print(f"Cloudy weather datasets - Daytime: {len(cloudy_daytime['frames'])}, Night: {len(cloudy_night['frames'])}, Dawn: {len(cloudy_dawn['frames'])}")

overcast_daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'overcast' and d['attributes']['timeofday_coarse'] == 'daytime'])
overcast_night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'overcast' and d['attributes']['timeofday_coarse'] == 'night'])
overcast_dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'overcast' and d['attributes']['timeofday_coarse'] == 'dawn/dusk'])
print(f"Overcast weather datasets - Daytime: {len(overcast_daytime['frames'])}, Night: {len(overcast_night['frames'])}, Dawn: {len(overcast_dawn['frames'])}")

foggy_daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'foggy' and d['attributes']['timeofday_coarse'] == 'daytime'])
foggy_night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'foggy' and d['attributes']['timeofday_coarse'] == 'night'])
foggy_dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'foggy' and d['attributes']['timeofday_coarse'] == 'dawn/dusk'])
print(f"Foggy weather datasets - Daytime: {len(foggy_daytime['frames'])}, Night: {len(foggy_night['frames'])}, Dawn: {len(foggy_dawn['frames'])}")

rainy_daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'rainy' and d['attributes']['timeofday_coarse'] == 'daytime'])
rainy_night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'rainy' and d['attributes']['timeofday_coarse'] == 'night'])
rainy_dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'rainy' and d['attributes']['timeofday_coarse'] == 'dawn/dusk'])
print(f"Rainy weather datasets - Daytime: {len(rainy_daytime['frames'])}, Night: {len(rainy_night['frames'])}, Dawn: {len(rainy_dawn['frames'])}")

Clear weather datasets - Daytime: 20800, Night: 9350, Dawn: 9650
Cloudy weather datasets - Daytime: 13850, Night: 6000, Dawn: 6750
Overcast weather datasets - Daytime: 7850, Night: 3250, Dawn: 3250
Foggy weather datasets - Daytime: 12900, Night: 7200, Dawn: 7450
Rainy weather datasets - Daytime: 20600, Night: 11500, Dawn: 9600


In [35]:
from os import makedirs, path

In [29]:
clear_daytime['frames'][0]

{'name': '00000000_img_front.jpg',
 'videoName': '0016-1b62',
 'intrinsics': {'focal': [640.0, 640.0], 'center': [640.0, 400.0], 'skew': 0},
 'extrinsics': {'location': [-7.529991149902344,
   219.90850830078125,
   1.7901159524917603],
  'rotation': [3.0515731904658596e-05,
   -0.06948350405481452,
   89.99990688190067]},
 'attributes': {'weather_coarse': 'clear',
  'timeofday_coarse': 'daytime',
  'weather_fine': 'clear',
  'timeofday_fine': 'morning/afternoon',
  'view': 'front',
  'town': '02',
  'sun_altitude_angle': '15',
  'cloudiness': '0.0',
  'precipitation': '0.0',
  'precipitation_deposits': '0.0',
  'wind_intensity': '10.0',
  'sun_azimuth_angle': '0.0',
  'fog_density': '0.1',
  'fog_distance': '0.75',
  'wetness': '0.0',
  'fog_falloff': '0.1'},
 'frameIndex': 0,
 'labels': [{'id': '1095',
   'attributes': {'type': 'vehicle.volkswagen.t2'},
   'category': 'bus',
   'box2d': {'x1': 593.0, 'y1': 397.0, 'x2': 612.0, 'y2': 415.0}},
  {'id': '1084',
   'attributes': {'type': 

In [33]:
for weather in ['clear', 'cloudy', 'overcast', 'foggy', 'rainy']:
    for time in ['daytime', 'night', 'dawn']:
        _id = f"{weather}_{time}"
        save_path = f"./data/SHIFT_SUBSET/{_id}/discrete/images/train/front"
        makedirs(save_path)
        dump(locals()[_id], open(path.join(save_path, "det_2d.json"), "w"))

In [40]:
for sett in ['train', 'val']:
    print("sett:", sett)
    data = load(open(f"./data/SHIFT/discrete/images/{sett}/front/det_2d.json"))

    for weather in ['clear', 'cloudy', 'overcast', 'foggy', 'rainy']:
        daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == weather and d['attributes']['timeofday_coarse'] == 'daytime'])
        night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == weather and d['attributes']['timeofday_coarse'] == 'night'])
        dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == weather and d['attributes']['timeofday_coarse'] == 'dawn/dusk'])
        print(f"{weather} weather datasets - Daytime: {len(daytime['frames'])}, Night: {len(night['frames'])}, Dawn: {len(dawn['frames'])}")

        for time in ['daytime', 'night', 'dawn']:
            _id = f"{weather}_{time}"
            save_path = f"./data/SHIFT_SUBSET/{_id}/discrete/images/{sett}/front"
            makedirs(save_path)
            dump(locals()[_id], open(path.join(save_path, "det_2d.json"), "w"))

sett: train
clear weather datasets - Daytime: 20800, Night: 9350, Dawn: 9650
cloudy weather datasets - Daytime: 13850, Night: 6000, Dawn: 6750
overcast weather datasets - Daytime: 7850, Night: 3250, Dawn: 3250
foggy weather datasets - Daytime: 12900, Night: 7200, Dawn: 7450
rainy weather datasets - Daytime: 20600, Night: 11500, Dawn: 9600
sett: val
clear weather datasets - Daytime: 2800, Night: 1200, Dawn: 1400
cloudy weather datasets - Daytime: 2400, Night: 1200, Dawn: 1250
overcast weather datasets - Daytime: 1600, Night: 700, Dawn: 550
foggy weather datasets - Daytime: 2650, Night: 1350, Dawn: 1350
rainy weather datasets - Daytime: 3200, Night: 1950, Dawn: 1400


In [41]:
for sett in ['train', 'val']:
    data = load(open(f"./data/SHIFT/discrete/images/{sett}/front/det_2d.json"))

    normal = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == 'clear' and d['attributes']['timeofday_coarse'] == 'daytime'])
    corrupted = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] != 'clear' or d['attributes']['timeofday_coarse'] != 'daytime'])
    print(f"weather datasets - Normal: {len(normal['frames'])}, Corrupted: {len(corrupted['frames'])}")

    for time in ['normal', 'corrupted']:
        _id = time
        save_path = f"./data/SHIFT_SUBSET/{_id}/discrete/images/{sett}/front"
        makedirs(save_path)
        dump(locals()[_id], open(path.join(save_path, "det_2d.json"), "w"))

weather datasets - Normal: 20800, Corrupted: 129200
weather datasets - Normal: 2800, Corrupted: 22200


In [6]:
from json import load

for tt in ["1x", "10x", "100x"]:
    print("tt:", tt)
    for sett in ['train', 'val']:
        print("sett:", sett)
        data = load(open(f"./data/SHIFT/continuous/images/{tt}/{sett}/front/det_2d.json"))

        for time_change in ['daytime_to_night']:
            clear = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == "clear" and d['attributes']['shift_type'] == time_change])
            rainy = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == "rainy" and d['attributes']['shift_type'] == time_change])
            cloudy = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == "cloudy" and d['attributes']['shift_type'] == time_change])
            foggy = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == "foggy" and d['attributes']['shift_type'] == time_change])
            overcast = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['weather_coarse'] == "overcast" and d['attributes']['shift_type'] == time_change])
            lengths = [len(clear['frames']), len(rainy['frames']), len(cloudy['frames']), len(foggy['frames']), len(overcast['frames'])]
            print(f"{time_change} datasets - Clear: {lengths[0]}, Rainy: {lengths[1]}, Cloudy: {lengths[2]}, Foggy: {lengths[3]}, Overcast: {lengths[4]}, Total: {sum(lengths)}/{len(data['frames'])}")

        for weather_change in ['clear_to_foggy', 'clear_to_rainy']:
            daytime = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['timeofday_coarse'] == "daytime" and d['attributes']['shift_type'] == weather_change])
            night = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['timeofday_coarse'] == "night" and d['attributes']['shift_type'] == weather_change])
            dawn = dict(config=data['config'], frames=[d for d in data['frames'] if d['attributes']['timeofday_coarse'] == "dawn/dusk" and d['attributes']['shift_type'] == weather_change])
            lengths = [len(daytime['frames']), len(night['frames']), len(dawn['frames'])]
            print(f"{weather_change} datasets - Daytime: {lengths[0]}, Night: {lengths[1]}, Dawn: {lengths[2]}, Total: {sum(lengths)}/{len(data['frames'])}")

    print()

tt: 1x
sett: train
daytime_to_night datasets - Clear: 960, Rainy: 560, Cloudy: 720, Foggy: 480, Overcast: 200, Total: 2920/9040
clear_to_foggy datasets - Daytime: 1800, Night: 520, Dawn: 560, Total: 2880/9040
clear_to_rainy datasets - Daytime: 1600, Night: 1000, Dawn: 640, Total: 3240/9040
sett: val
daytime_to_night datasets - Clear: 160, Rainy: 280, Cloudy: 200, Foggy: 80, Overcast: 160, Total: 880/2280
clear_to_foggy datasets - Daytime: 240, Night: 200, Dawn: 240, Total: 680/2280
clear_to_rainy datasets - Daytime: 360, Night: 240, Dawn: 120, Total: 720/2280

tt: 10x
sett: train
daytime_to_night datasets - Clear: 3200, Rainy: 4000, Cloudy: 2800, Foggy: 2800, Overcast: 1600, Total: 14400/40616
clear_to_foggy datasets - Daytime: 4800, Night: 2616, Dawn: 4400, Total: 11816/40616
clear_to_rainy datasets - Daytime: 6800, Night: 3200, Dawn: 4400, Total: 14400/40616
sett: val
daytime_to_night datasets - Clear: 1012, Rainy: 0, Cloudy: 1457, Foggy: 400, Overcast: 0, Total: 2869/10069
clear_to_